In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv


def load_data(file_path):
    # 读取CSV文件
    # read the CSV file
    df = pd.read_csv(file_path)
    # 获取时间t, x位置和y位置，并转换为Tensor，同时确保数据类型为torch.float32
    # Retrieve time t, x, y, and convert them to Tensors, ensuring that the data type is torch.float32
    t_obs = torch.tensor(df['Time'].values, dtype=torch.float32).view(-1, 1)
    x_obs = torch.tensor(df['X'].values, dtype=torch.float32).view(-1, 1)
    y_obs = torch.tensor(df['Y'].values, dtype=torch.float32).view(-1, 1)
    t_start = t_obs[0].detach().numpy().item()  # 训练集数据和测试集数据的时间都是从0开始的
    t_end = t_obs[-1].detach().numpy().item()

    return t_start, t_end, t_obs, x_obs, y_obs

# 定义重力加速度
# define g
g = 9.8

nodes = 32
# 定义神经网络，输入为时间t，输出为轨迹信息x, y坐标以及阻力参数k。
# 你可以采用别的定义方法，这里只是给出了一种可行的方式
# Define the neural network, with input as time t , and output as trajectory information including  x  and  y  coordinates, as well as the drag parameter k .
# You can use a different method; this is just one possible approach.

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.hidden_layer1 = nn.Linear(1, nodes)

        self.output_layer_x = nn.Linear(nodes, 1)
        self.output_layer_y = nn.Linear(nodes, 1)
        self.output_layer_k = nn.Linear(nodes, 1)

    def forward(self, t):
        outputs = torch.tanh(self.hidden_layer1(t))

        x = self.output_layer_x(outputs)
        y = self.output_layer_y(outputs)
        k = self.output_layer_k(outputs)
        return x, y, k

# 定义损失函数，包括物理损失与数据损失
# Define the loss function, including physical loss and data loss.

def loss(t, net, t_obs, x_obs, y_obs, epoch):
    x, y, k_pred = net(t)

    # 计算数据损失
    # data loss
    x_train_pred, y_train_pred, _ = net(t_obs)
    total_loss = torch.mean((x_train_pred - x_obs)**2)

    return total_loss

def export_parameter(net, submission_path):
    # 阻力系数，由你通过训练得出
    # Drag coefficient, derived from your training.
    k_pred = 1.0
    # 最大高度，由你通过训练得出
    # Maximum height, derived from your training.
    height_max = 1.0
    # 最远射程，由你通过训练得出
    # Range, derived from your training.
    range_max = 1.0
    with open(submission_path, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['k', 'height_max', 'range_max'])
        writer.writerow([k_pred, height_max, range_max])

def calculate_parameters(data_path, submission_path):
    t_start, t_end, t_obs, x_obs, y_obs = load_data(data_path)

    net = Net()
    export_parameter(net, submission_path)

In [ ]:
#---------提交范例-------#
#---------Submission example-------#
import zipfile
import os
#导入训练集数据
# import training data

TRAIN_PATH = "/bohr/train-g6oc/v1/"
calculate_parameters(data_path = TRAIN_PATH + "projectile_train.csv", submission_path = "submission_train.csv") #求解训练集方程

In [ ]:
# 导入测试集数据
#“DATA_PATH”是测试集加密后的环境变量，按照如下方式可以在提交后，系统评分时访问测试集，但是选手无法直接下载
# import test data
#"DATA_PATH" is an encrypted environment variable for the test set. It can be accessed by the system for scoring after submission, but contestants cannot directly download it.

if os.environ.get('DATA_PATH'):
    DATA_PATH = os.environ.get("DATA_PATH") + "/"
else:
    #Baseline运行时，因为无法读取测试集，所以会有此条报错，属于正常现象
    #During the execution of the Baseline, since it is unable to read the test set, this error message will appear, which is a normal phenomenon
    print("During the execution of the Baseline, since it is unable to read the test set, this error message will appear, which is a normal phenomenon.")
calculate_parameters(data_path = DATA_PATH + "projectile_testA.csv", submission_path = "submission_testA.csv") #求解Public测试集方程
calculate_parameters(data_path = DATA_PATH + "projectile_testB.csv", submission_path = "submission_testB.csv") #求解Private测试集方程

In [ ]:
# 定义要打包的文件和压缩文件名
files_to_zip = ['submission_testA.csv', 'submission_testB.csv']
zip_filename = 'submission.zip'

# 创建一个 zip 文件
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    for file in files_to_zip:
        # 将文件添加到 zip 文件中
        zipf.write(file, os.path.basename(file))

print(f'{zip_filename} 创建成功!')